In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import tifffile
import cv2
import random
import datetime
import time

from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QColor, QFont, QPainterPath, QBrush
from PyQt5.QtCore import Qt, QTimer, QItemSelection
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from optic.config import *
from optic.controls import *
from optic.manager import *
from optic.gui import *
from optic.io import *
from optic.utils import *
from optic.preprocessing import *
from optic.visualization import *

class Suite2pROITrackingGUI(QMainWindow):
    def __init__(self):
        APP_NAME = "SUITE2P_ROI_TRACKING"
        QMainWindow.__init__(self)
        self.widget_manager, self.config_manager, self.data_manager, self.control_manager = initManagers(WidgetManager(), ConfigManager(), DataManager(), ControlManager())
        self.config_manager.setCurrentApp(APP_NAME)
        self.app_keys = self.config_manager.gui_defaults["APP_KEYS"]
        key_app = self.app_keys[0]
        self.app_key_sec = self.app_keys[1]

        self.setupUI_done = False
        setupMainWindow(self, self.config_manager.gui_defaults)

        self.initUI()

    """
    setup UI Function
    """
    def initUI(self):
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout_main = QGridLayout(self.central_widget)

        # FileLoadUI用のレイアウト
        self.layout_file_load = QVBoxLayout()
        self.setupFileLoadUI()
        self.layout_main.addLayout(self.layout_file_load, 1, 0, 1, 1)

        # メインUI用のレイアウト
        self.layout_main_ui = QGridLayout()
        self.layout_main.addLayout(self.layout_main_ui, 0, 0, 1, 1)

    def setupFileLoadUI(self):
        file_load_widget = QWidget()
        layout = QVBoxLayout(file_load_widget)
        # ファイル読み込み用のUIを追加
        layout.addLayout(self.makeLayoutSectionBottom())
        # bindFunc
        self.bindFuncFileLoadUI()

        self.layout_file_load.addWidget(file_load_widget)

    def loadFilePathsandInitialize(self):
        self.control_manager, self.data_manager = initManagers(self.control_manager, self.data_manager)
        success = self.loadData()
        if success:
            self.setupMainUI()
        else:
            return

    def setupMainUI(self):
        if self.setupUI_done:
            # メインUIのクリア
            clearLayout(self.layout_main_ui)
        
        # 新しいメインUIの設定
        self.setupMainUILayouts()
        self.setupControls()
        self.bindFuncAllWidget()

        self.setupUI_done = True

    def loadData(self):
        return True

    def setupMainUILayouts(self):
        self.layout_main_ui.addLayout(self.makeLayoutSectionLeftUpper(), 0, 0, 1, 1)
        self.layout_main_ui.addLayout(self.makeLayoutSectionRightUpper(), 0, 1, 1, 1)
        self.layout_main_ui.addLayout(self.makeLayoutSectionCentral(), 1, 0, 1, 2)

    def setupControls(self):
        pass
        # self.control_manager.table_controls[key_app] = TableControl(
        #                                                         key_app=key_app,
        #                                                         q_table=self.widget_manager.dict_table[key_app],
        #                                                         data_manager=self.data_manager,
        #                                                         widget_manager=self.widget_manager,
        #                                                         config_manager=self.config_manager,
        #                                                         control_manager=self.control_manager,
        #                                                         )
        
        # self.control_manager.table_controls[key_app].setupWidgetROITable(key_app)
        # self.control_manager.view_controls[key_app] = ViewControl(
        #                                                             key_app=key_app,
        #                                                             q_view=self.widget_manager.dict_view[key_app], 
        #                                                             q_scene=self.widget_manager.dict_scene[key_app], 
        #                                                             data_manager=self.data_manager, 
        #                                                             widget_manager=self.widget_manager,
        #                                                             config_manager=self.config_manager,
        #                                                             control_manager=self.control_manager,
        #                                                             )
        # self.control_manager.view_controls[key_app].setViewSize()

    """
    makeLayout Function; Component
    小要素のLayout
    return -> Layout
    """
    "Left/Right Upper"
    # Table, ROI count label
    def makeLayoutComponentTable(self, key_app):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutTableROICountLabel(self.widget_manager, 
                                                      key_label=key_app, 
                                                      key_table=key_app, 
                                                      table_columns=self.config_manager.table_columns[key_app].getColumns()))
        return layout
    
    # View
    def makeLayoutComponentView(self, key_app):
        layout = QVBoxLayout()
        layout.addWidget(self.widget_manager.makeWidgetView(key=key_app))
        return layout
    
    # Table + View
    def makeLayoutComponentTable_View(self, key_app):
        layout = QHBoxLayout()
        layout.addLayout(self.makeLayoutComponentView(key_app))
        layout.addLayout(self.makeLayoutComponentTable(key_app))
        return layout

    # ROI property label, threshold lineedit
    def makeLayoutComponentROIPropertyDisplay_Threshold(self, key_app):
        layout = QVBoxLayout()

        layout.addLayout(makeLayoutROIProperty(self.widget_manager, key_label=f"{key_app}_roi_prop"))
        layout.addLayout(makeLayoutROIThresholds(
            self.widget_manager, 
            key_label=f"{key_app}_roi_threshold", 
            key_lineedit=f"{key_app}_roi_threshold", 
            key_checkbox=f"{key_app}_roi_threshold", 
            label_checkbox="ROI Show Threshold", 
            list_threshold_param=["npix", "compact"]))
        return layout
    
    # ROI display, background image button group, checkbox
    def makeLayoutComponentROIDisplay_BGImageDisplay(self, key_app):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutDislplayCelltype(self, 
                                                  self.widget_manager, 
                                                  key_buttongroup=f'{key_app}_display_celltype', 
                                                  table_columns=self.config_manager.table_columns[key_app].getColumns()))
        layout.addLayout(makeLayoutBGImageTypeDisplay(self, 
                                                      self.widget_manager, 
                                                      key_buttongroup=f'{key_app}_im_bg_type'))
    
    # channel contrast, ROI opacity slider
    def makeLayoutComponentContrastOpacitySlider(self, key_app):
        layout = QVBoxLayout()
        channels = self.config_manager.gui_defaults["CHANNELS"]
        layout_channel = QHBoxLayout()
        for channel in channels:
            layout_channel.addLayout(makeLayoutContrastSlider(self.widget_manager, 
                                                            key_label=f"{key_app}_{channel}", 
                                                            key_checkbox=f"{key_app}_{channel}", 
                                                            key_slider=f"{key_app}_{channel}", 
                                                            label_checkbox=f"Show {channel} channel", 
                                                            label_label=f"{channel} Value", 
                                                            checked=True))

        layout.addLayout(layout_channel)
        layout.addLayout(makeLayoutOpacitySlider(self.widget_manager, 
                                                key_label=key_app, 
                                                key_slider=key_app, 
                                                label=key_app
                                                ))
        return layout
    
    # Table, ROI count label, Set ROI Celltype, ROICheck IO
    def makeLayoutComponentTable_ROICountLabel_ROISetSameCelltype_ROICheckIO(self, key_app):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutAllROISetSameCelltype(self.widget_manager, 
                                                         key_button=key_app, 
                                                         table_columns=self.config_manager.table_columns[key_app].getColumns()))
        layout.addLayout(makeLayoutROICheckIO(self.widget_manager, 
                                              key_button=key_app))
        return layout

    # ROI Filter, threshold
    def makeLayoutComponentROIFilter(self, key_app):
        layout = QHBoxLayout()
        layout.addLayout(makeLayoutROIFilterThreshold(self.widget_manager, key_label=key_app, key_lineedit=key_app, dict_roi_threshold=self.config_manager.gui_defaults["ROI_THRESHOLDS"]))
        layout.addLayout(makeLayoutROIFilterButton(self.widget_manager, key_label=key_app, key_button=key_app))
        return layout

    def makeLayoutComponent_Label_Radiobutton_Checkbox_Slider(self, key_app):
        layout = QVBoxLayout()
        layout.addLayout(self.makeLayoutComponentROIPropertyDisplay_Threshold(key_app))
        layout.addLayout(self.makeLayoutComponentROIDisplay_BGImageDisplay(key_app))
        layout.addLayout(self.makeLayoutComponentContrastOpacitySlider(key_app))
        return layout
    
    def makeLayoutComponent_ROICheckButton_Lineedit(self, key_app):
        layout = QVBoxLayout()
        layout.addLayout(self.makeLayoutComponentTable_ROICountLabel_ROISetSameCelltype_ROICheckIO(key_app))
        layout.addLayout(self.makeLayoutComponentROIFilter(key_app))
        return layout
    
    def makeLayoutComponent_Under_Table_View(self, key_app):
        layout = QHBoxLayout()
        layout.addLayout(self.makeLayoutComponent_Label_Radiobutton_Checkbox_Slider(key_app))
        layout.addLayout(self.makeLayoutComponent_ROICheckButton_Lineedit(key_app))
        return layout

    "Central"
    # ROI Matching Method
    def makeLayoutComponentROIMatchingMethod(self):
        layout = QVBoxLayout()
        layout_label = QHBoxLayout()
        layout_label.addWidget(self.widget_manager.makeWidgetLabel(key="roi_matching_method", label="ROI Matching Method"))
        layout_button = QHBoxLayout()
        layout_button.addWidget(self.widget_manager.makeWidgetComboBox(key="roi_matching_method", items=self.config_manager.gui_defaults["ROI_MATCHING_METHOD"]))
        layout_button.addWidget(self.widget_manager.makeWidgetButton(key="roi_matching", label="ROI Match"))
        layout_button.addWidget(self.widget_manager.makeWidgetButton(key="config_elastix", label="Elastix Config"))
        layout_button.addWidget(self.widget_manager.makeWidgetButton(key="roi_tracking", label="ROI Tracking"))
        layout_button.addWidget(self.widget_manager.makeWidgetButton(key="roi_tracking_export", label="Export Tracking Result"))
        layout.addLayout(layout_label)
        layout.addLayout(layout_button)
        return layout

    "Bottom"
    # ファイル読み込み用UI Layout
    def makeLayoutComponentFileLoadUI(self):
        layout = QVBoxLayout()

        # LineEdit
        for key_app in self.app_keys:
            list_label = [f"Fall mat file path ({key_app} Image)", f"Reference Tiff image file path ({key_app} Image) (optional)"]
            list_key = [f"{key_app}_path_fall", f"{key_app}_path_reftif"]
            for label, key in zip(list_label, list_key):
                layout.addLayout(makeLayoutLoadFileWidget(self.widget_manager, label=label, key_label=key, key_lineedit=key, key_button=key))
        # Button
        layout.addLayout(makeLayoutLoadFileExitHelp(self.widget_manager))
        return layout

    

    """
    makeLayout Function; Section
    領域レベルの大Layout
    """
    # 上段, 左
    def makeLayoutSectionLeftUpper(self):
        layout = QVBoxLayout()
        layout.addLayout(self.makeLayoutComponentTable_View(self.app_keys[0]))
        layout.addLayout(self.makeLayoutComponent_Under_Table_View(self.app_keys[0]))
        return layout
    
    # 上段. 右
    def makeLayoutSectionRightUpper(self):
        layout = QVBoxLayout()
        layout.addLayout(self.makeLayoutComponentTable_View(self.app_keys[1]))
        layout.addLayout(self.makeLayoutComponent_Under_Table_View(self.app_keys[1]))
        return layout
    
    # 中段
    def makeLayoutSectionCentral(self):
        layout = QHBoxLayout()
        layout.addLayout(self.makeLayoutComponentROIMatchingMethod())
        return layout

    # 下段
    def makeLayoutSectionBottom(self):
        layout = self.makeLayoutComponentFileLoadUI()
        return layout

    """
    bindFunc Function
    配置したwidgetに関数を紐づけ
    """
    def bindFuncFileLoadUI(self):
        for key_app in self.app_keys:
            list_key = [f"{key_app}_path_fall", f"{key_app}_path_reftif"]
            list_filetype = ["mat", "tiff"]
            for key, filetype in zip(list_key, list_filetype):
                bindFuncLoadFileWidget(q_widget=self, q_button=self.widget_manager.dict_button[key], q_lineedit=self.widget_manager.dict_lineedit[key], filetype=filetype)

        self.widget_manager.dict_button["load_file"].clicked.connect(lambda: self.loadFilePathsandInitialize())
        bindFuncExit(q_window=self, q_button=self.widget_manager.dict_button["exit"])

    def bindFuncAllWidget(self):
        pass


if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
    applyAppStyle(app)
    gui = Suite2pROITrackingGUI()
    gui.show()
    sys.exit(app.exec_())